In [2]:
from tensorflow.keras.layers import Input,LSTM,Dense
from tensorflow.keras.models import Model
import numpy as np

In [35]:
batch_size=64
epochs=100
latent_dim=256
num_samples=10000 # el data kolha 12k ana ha4t8l 3la 10k mnha bs el ahsan a4t8l 3al 12k kolohm
data_path = 'ara.txt'

In [36]:
input_texts = [] #eng
target_texts = [] # arabic b7ot ay 3lama lldecoder 34an msln aaolo awl ma t4of space tab msln ebd2 e3ml translation
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n") #new line \n
for line in lines[: min(num_samples, len(lines) - 1)]: # 3mlt minus 1 34an el data 12412 w 3ndy lma 4oftha tl3t 12413 min (10k aw 12412)
    input_text, target_text, _ = line.split("\t") #ha3ml split 34an el input lwhdo w el target lwhdo 
    # We use "tab" as the "start sequence" character input englisch target arabic
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n" # awl ma el decoder yla2y /t hybd2 decoding
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_characters

{' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '€'}

In [9]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '9',
 ':',
 '?',
 'F',
 'M',
 'P',
 '\\',
 'e',
 'f',
 'i',
 'o',
 'r',
 'x',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'إ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ـ',
 'ف',
 'ق',
 'ك',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ى',
 'ي',
 'ً',
 'ٌ',
 'ٍ',
 'َ',
 'ُ',
 'ِ',
 'ّ',
 'ْ',
 '٠',
 '١',
 '٢',
 '٣',
 '٤',
 '٥',
 '٦',
 '٨',
 '٩',
 'ٱ',
 'ی',
 '۰',
 '۱',
 '۹',
 '\u200d',
 '\u200f'}

In [11]:
target_texts

['\tمرحبًا.\n',
 '\tاركض!\n',
 '\tاخفض رأسك!\n',
 '\tاخفضي رأسك!\n',
 '\tاخفضوا رؤوسكم!\n',
 '\tالنجدة!\n',
 '\tاقفز!\n',
 '\tقف!\n',
 '\tتوقف !\n',
 '\tإنتظر\n',
 '\tداوم.\n',
 '\tاستمر.\n',
 '\tمرحباً.\n',
 '\tأهلاً.\n',
 '\tمرحباً.\n',
 '\tتعجّل!\n',
 '\tاستعجل!\n',
 '\tانا اري\n',
 '\tأنا فُزت!\n',
 '\tاسترح.\n',
 '\tابتسم.\n',
 '\tفي صحتك.\n',
 '\tهل فهمت؟\n',
 '\tركض.\n',
 '\tأعرف.\n',
 '\tأعلم ذلك.\n',
 '\tأنا أعلم\n',
 '\tأنا في 19\n',
 '\tأنا بخير.\n',
 '\tاستمع\n',
 '\tغير معقول!\n',
 '\tحقاً؟\n',
 '\tشكراً!\n',
 '\tشكرا.\n',
 '\tلماذا أنا؟\n',
 '\tرائع!\n',
 '\tخذ راحتك.\n',
 '\tأغرب عن وجهي\n',
 '\tهاتفني.\n',
 '\tاتصل بي.\n',
 '\tتفضل بالدخول.\n',
 '\tتعال إلى الداخل\n',
 '\tبالله عليك!\n',
 '\tهيا\n',
 '\tهيّا\n',
 '\tاخرج من هنا!\n',
 '\tأُخرج!\n',
 '\tاخرج!\n',
 '\tاتركني و شأني.\n',
 '\tاذهب بعيداً.\n',
 '\tارحل.\n',
 '\tمع السلامة.\n',
 '\tإلى اللقاء\n',
 '\tإنتظر\n',
 '\tلقد أتى.\n',
 '\tهو يجري\n',
 '\tساعدني!\n',
 '\tالنجدة! ساعدني!\n',
 '\tساعدوني\n',
 '\tانتظر.\n

In [37]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) #unique car in eng
num_decoder_tokens = len(target_characters) #unique car in arabic
max_encoder_seq_length = max([len(txt) for txt in input_texts]) #atwl gomla eng
max_decoder_seq_length = max([len(txt) for txt in target_texts]) #atwl gomla arab

In [13]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 36
Max sequence length for outputs: 54


In [38]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) #lazem a7wlha lvectors key = char value = index
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [15]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  "'": 3,
  ',': 4,
  '-': 5,
  '.': 6,
  '0': 7,
  '1': 8,
  '2': 9,
  '3': 10,
  '4': 11,
  '5': 12,
  '6': 13,
  '7': 14,
  '8': 15,
  '9': 16,
  ':': 17,
  '?': 18,
  'A': 19,
  'B': 20,
  'C': 21,
  'D': 22,
  'E': 23,
  'F': 24,
  'G': 25,
  'H': 26,
  'I': 27,
  'J': 28,
  'K': 29,
  'L': 30,
  'M': 31,
  'N': 32,
  'O': 33,
  'P': 34,
  'Q': 35,
  'R': 36,
  'S': 37,
  'T': 38,
  'U': 39,
  'V': 40,
  'W': 41,
  'X': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  '€': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '"': 4,
  '(': 5,
  ')': 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '/': 10,
  '0': 11,
  '1': 12,
  '2': 13,
  '3': 14,
  '5': 15,
  '6': 16,
  '7': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'F'

In [39]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),#(num pairs,max_eng_sent,num eng char)change text to vectors
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),#(num pairs,max_arab_sent,num arab char)
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)


In [40]:
encoder_input_data.shape #(num pairs,max_eng_sent,num eng char)

(10000, 36, 71)

In [41]:
decoder_input_data.shape#(num pairs,max_arab_sent,num eng char)

(10000, 54, 94)

In [30]:
decoder_target_data.shape

(10000, 54, 94)

In [31]:
input_token_index['H']

26

In [42]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)): # word w=1000 o=0100 ONE HOT ENCODING 
    for t, char in enumerate(input_text): # t=time eli hoa awl input lllstm bta3ty
        encoder_input_data[i, t, input_token_index[char]] = 1.0 # eli hoa el 3d raay eli lsa 3mlo fo2 b3ml fill lkol char b one
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 #padding 34an el spaces by7ot mkanha wahed bylf 3la kolo brdo
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0 #decoder input data
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0 #3ayz yn2s wahed tab leh 34an awl haga btd5ol lldecodder heya el space aw el tab
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0 #padding 34an el spaces by7ot mkanha wahed bylf 3la kolo brdo
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0 #padding 34an el spaces by7ot mkanha wahed bylf 3la kolo brdo

In [43]:
import tensorflow as tf #MODEL BUILDING

encoder_inputs = tf.keras.Input(shape=(None, num_encoder_tokens)) #none 34an heya mot8yra m4 ha2dr a7dd kam bzbt ,3dd el unique charcters
encoder = tf.keras.layers.LSTM(latent_dim, return_state=True) #return state eli heya el hidden state  c^t aw a^t activation state
encoder_outputs, state_h, state_c = encoder(encoder_inputs) #output,hidden state,ceil state ha5od w harg3o lldecoder ka input

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = tf.keras.Input(shape=(None, num_decoder_tokens)) ##none 34an heya mot8yra m4 ha2dr a7dd kam bzbt ,3dd el unique charcters

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) #(el code eli fo2k bzbt)
decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation="softmax") #bnst5dm softmax 34an el arg max w nbdl el number beltexst nafso 34an kda bnzwd dense layes w n7ot 3dd el neurons bldecoder tokens eli heya 3dd el 7rof b el lo8a el 3rbya
decoder_outputs = decoder_dense(decoder_outputs) #el decoder output ha5do w hab3to llsoft max 34an y3mli classification

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [20]:
import tensorflow as tf #TRAIN MODEL

# Compile the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]) 

# Train the model
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2, #20% mn data el training fe el validation 
)

# Save the model
model.save("s2s_model.h5")



Epoch 1/100


125/125 [==============================] - 16s 101ms/step - loss: 1.3766 - accuracy: 0.7023 - val_loss: 1.7124 - val_accuracy: 0.5841
Epoch 2/100
125/125 [==============================] - 11s 88ms/step - loss: 1.1170 - accuracy: 0.7155 - val_loss: 1.5734 - val_accuracy: 0.5795
Epoch 3/100
125/125 [==============================] - 12s 95ms/step - loss: 1.0386 - accuracy: 0.7303 - val_loss: 1.4668 - val_accuracy: 0.6092
Epoch 4/100
125/125 [==============================] - 11s 87ms/step - loss: 0.9625 - accuracy: 0.7464 - val_loss: 1.3755 - val_accuracy: 0.6359
Epoch 5/100
125/125 [==============================] - 11s 88ms/step - loss: 0.9219 - accuracy: 0.7584 - val_loss: 1.3296 - val_accuracy: 0.6503
Epoch 6/100
125/125 [==============================] - 12s 99ms/step - loss: 0.8859 - accuracy: 0.7646 - val_loss: 1.2795 - val_accuracy: 0.6627
Epoch 7/100
125/125 [==============================] - 11s 87ms/step - loss: 0.8644 - accuracy: 0.7690 - val_loss: 1.2703 - va

C:\Users\Youssef\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
import tensorflow as tf
import numpy as np

# Load the model
model = tf.keras.models.load_model("s2s_model.h5")

# Extract encoder and decoder parts
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = tf.keras.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [34]:
# You can now generate decoded sentences as such:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


1/1 [==============================] - 0s 18ms/step
-
Input sentence: Hi.
Decoded sentence: افعلم!

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Run!
Decoded sentence: افعل ذلك!

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Duck!
Decoded sentence: افهل ما وريدة.

1/1 [==============================] - 0s 17ms/step
-
Input sentence: Duck!
Decoded sentence: افهل ما وريدة.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Duck!
Decoded sentence: افهل ما وريدة.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Help!
Decoded sentence: استمر.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Jump!
Decoded sentence: استمر.

1/1 [==============================] - 0s 18ms/step
-
Input sentence: Stop!
Decoded sentence: استمر.

1/1 [==============================] - 0s 19ms/step
-
Input sentence: Stop!
Decoded sentence: استمر.

1/1 [==============================] - 0s 18ms/step
-
Inpu